# 미디언 필터링
- 미디언이라는 용어처럼 중간값을 나타낸다, 
- 일정 여역에서 다른 화소드로가 밝기가 심하게 차이가 나는 화소들은 임펄스 잡음(impluse noise) 이나 소금 - 후추 잡음(salt & pepper noise)일 가능성이 높다. 미디언 필터링 과정에서 마스크 영역 내의 심하게 차이가 나는 화소들은 정렬로 인해서 최하위 값이나 최상위 값이 된다. 미디언 필터링에서 중간값 이외의 다른 나머지 값들은 출력 화소로 지정되지 않고 제거된다. 이러한 이유로 미디언 필터링은 임펄스 잡음이나 소금-후추 잡음을 잘 제거해 준다. 그리고 평균 값 필터링에 비하면 블러링 현상이 적다 
- 다만, 마스크의 크기가 커지면 잡음 제거 성능은 향상되지만, 모든 화소를 순회하며 마스크 범위에 대해 정렬 알고리즘을 수행해야 하는 부담 대문에 수행 시간이 기하급수적으로 증가한다. 
    - 임펄스 잡음 : 0 또는 255의 픽셀 값과 같이 두력하게 잘못된 픽셀 값을 갖는 noise
    - 소금 -후추 잡음 : 영상 내에서 검은색 또는 흰색 점의 형태로 발생하는 noise

In [1]:
import numpy as np, cv2

In [2]:
def median_filter(image, ksize):
    rows, cols = image.shape[:2]
    dst = np.zeros((rows, cols), np.uint8)
    center = ksize // 2                                 # 마스크 절반 크기

    for i in range(center, rows - center):              # 입력 영상 순회
        for j in range(center, cols - center):
            y1, y2 = i - center, i + center + 1             # 마스크 높이 범위
            x1, x2 = j - center, j + center + 1             # 마스크 너비 범위
            mask = image[y1:y2, x1:x2].flatten()            # 마스크 영역

            sort_mask = cv2.sort(mask, cv2.SORT_EVERY_COLUMN)    # 정렬 수행
            dst[i, j] = sort_mask[sort_mask.size//2]                  # 출력화소로 지정
    return dst

In [3]:
def salt_pepper_noise(img, n):
    h, w = img.shape[:2]
    x, y = np.random.randint(0, w, n), np.random.randint(0, h, n)
    noise = img.copy()
    for (x,y) in zip(x,y):
        noise[y, x] = 0 if np.random.rand() < 0.5 else 255
    return noise


In [5]:
image = cv2.imread("img/median2.jpg", cv2.IMREAD_GRAYSCALE)
if image is None: raise Exception("영상파일 읽기 오류")

In [6]:
noise = salt_pepper_noise(image, 500)
med_img1 = median_filter(noise, 3)                            # 사용자 정의 함수
med_img2 = cv2.medianBlur(noise, 3)                          # OpenCV 제공 함수

In [7]:
cv2.imshow("image", image),
cv2.imshow("noise", noise),
cv2.imshow("median - User", med_img1)
cv2.imshow("median - OpenCV", med_img2)
cv2.waitKey(0)

-1

![result](img/11_result.png)